In [64]:
from diffusers import StableDiffusionPipeline
from huggingface_hub import notebook_login
from pathlib import Path
import torch


if not (Path.home() / ".huggingface" / "token").exists():
    notebook_login()

In [65]:
model_id = "runwayml/stable-diffusion-v1-5"
images_dir = ".images/"
device = "cuda" if torch.cuda.is_available() else "mps" if torch.has_mps else "cpu"

pipe = StableDiffusionPipeline.from_pretrained(model_id)
pipe = pipe.to(device)

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


In [ ]:
# https://huggingface.co/docs/diffusers/optimization/mps#inference-pipeline
# Recommended if your computer has < 64 GB of RAM
pipe.enable_attention_slicing()

In [ ]:
prompt = "a photo of an astronaut riding a horse on mars"

# First-time "warmup" pass (see explanation above)
# Ref: https://huggingface.co/docs/diffusers/optimization/mps#inference-pipeline
_ = pipe(prompt, num_inference_steps=1)

image = pipe(prompt).images[0]
image

In [61]:
# Check for existing filename and append a number if necessary
# to avoid overwriting existing files.

def image_name(prompt, i):
    return prompt.replace(" ", "_") + f"_{i}.png"

i = 1
while Path(images_dir + image_name(prompt, i)).exists():
    i += 1

path = Path(images_dir + image_name(prompt, i))
image.save(path)

print(f"Saved image to {path}")

Saved image to .images/a_photo_of_an_astronaut_riding_a_horse_on_mars_2.png
